## Блокнот для кроссвалидации
Из-за ограничений по объему памяти и времени выполнения  проверяем только лучший  вариант

In [1]:
from task import * 
# from utils import X,y,cat_cols,fillna,normalize,skew,encode

## Загрузка данных

In [2]:
def data():
    return ( pd.read_csv('datasets/train.csv') )

def cross_valid_sets():
    df0 = data()
    df0,df2 = train_test_split(df0,test_size=.5)
    df0,df1 = train_test_split(df0,test_size=.5)
    df2,df3 = train_test_split(df2,test_size=.5)
    return([df0,df1,df2,df3])


### Класс для кроссвалидации коррекций данных и гиперпараметров
 

In [3]:
class CrossValidator(Task):

    def __init__(self, dfs,params = {}):
        super().__init__(dfs,params)
        self.datasets = []
        for i in [0,1,2,3]:
            dx = shuffle(pd.concat( dfs[:i]+dfs[(i+1):] ,axis=0 )).copy()
            self.datasets.append(ScoreSet( dx, dfs[i].copy(),params=self.params) ) 
# Training dataset  combined from 4 base dataframes to 4 folds in all possible combinations 3:1  

    def process_fold(self,f):
        self.log +=(f"{f.log} \n")  
        f.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        f.prepare().fit().predict()
        self.log +=(f"{f.log} \n")  
# Then run validation on each fold 
        
    def process(self):
        for f in self.datasets: 
            self.process_fold(f) 
        logging.debug(f"cross-validation finished: {self.log}")
        self.score = self.mean_score()      
        print(f"MAPE mean: {self.score}")
        logging.info(f"MAPE mean: {self.score}")        
        return(self)
    
    def mean_score(self):
        return sum( [ds.scores() for ds in self.datasets ])/len(self.datasets)
    

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️
</div>

> Не могу не отметить высокое качество написание кода, но почему мы не смогли реализовать решение кросс-валидации с применением стандартной библиотеки Scikit Learn? Если принимаешь решение использовать свою кастомную кросс-валидацию очень неплохо бы получить комментарии по целесообразности этого вороса.
    

In [4]:
params = { 
        'rounding': {'odometer': 1000, 'condition': 1, 'prefix_size': 10}, 
        'model': {'criterion': 'poisson', 'n_estimators': 64, 'max_depth': 32, 'min_samples_leaf': 2, 'max_features': 16},
        'slice': {
            'cat_cols': ['make', 'model', 'trim', 'body', 'transmission', 'state', 'seller', 'year', 'odometer', 'condition']
                }, 
        'model_class': RandomForestRegressor,
        'encoder_class': TargetEncoder,
        'encoder': {'min_samples_leaf': 2, 'smoothing': 5},
        'sellers': {'stages': 6}
        }
cv = CrossValidator( cross_valid_sets() ,params=params)
cv.process()
import time
time.sleep(10)
del cv

MAPE values from log </br>
2023-06-01 20:43:16,109 - INFO:  model: MAPE= 0.1321936717184024  </br>
2023-06-01 20:48:28,045 - INFO:  model: MAPE= 0.13226218671855258  </br>
2023-06-01 20:54:02,861 - INFO:  model: MAPE= 0.13184536203222288  </br>
2023-06-01 20:59:39,121 - INFO:  model: MAPE= 0.13037589789014575  </br>
<b>MAPE MEAN 0.131669279589831</b> </br>
Результат предварительного теста в общем подтвреждён кросс-валидацией (при вдвое меньшем числе деревьев)

In [5]:
import gc
gc.collect()